# GENIE Learning Scenarios with Carbon Taxes [TEST]

**Pre-requisites**
- You have the *MESSAGEix* framework installed and working
- You have run Westeros baseline scenario (``westeros_baseline.ipynb``) and solved it successfully

In [1]:
import ixmp
import message_ix
import numpy as np
import pandas as pd
import yaml

from collections.abc import Mapping
from itertools import repeat
from message_ix.models import MESSAGE_ITEMS
from message_ix.utils import make_df

from message_ix.tools.add_learning import add_learning


%matplotlib inline

<IPython.core.display.Javascript object>

In [2]:
mp = ixmp.Platform("local", jvmargs=['-Xmx8G'])

## Making a clone of the existing scenario 'baseline'

Dump scenario to excel nad create a scenario from excel

In [3]:
base = message_ix.Scenario(
    mp, model="GENIE", scenario="learning",
    )

Clone baseline scenario and add learning parameters

## High CO2 Tax Scenarios

Here, we run the model under a range of CO2 tax scenaros

In [4]:
# Firstly, let's create an empty list to contain the emissions data
tax_growth = {'n':0,'l':0.025,'m':0.050,'h':0.075}
data_collect = {par: [] for par in list(tax_growth.keys())}


In [5]:
# Scenario setup
txs = 'n'
lrn = 'y'
rnd = 'n'

Base2020 = 0 if txs == 'n' else 10 # $/tCO2
lrn_val = 1 if lrn == 'y' else 0

In [6]:
scen = base.clone(
    "GENIE",
    "with_"+lrn+"learning_CT"+txs+"-RNDn",
    txs+" carbon tax with"+rnd+"RND target",
    keep_solution=False,
    )
scen.check_out()

all_years = scen.set('year')
all_nodes = scen.set('node')
type_years = list(all_years[all_years>=2020])
nodes = list(all_nodes[all_nodes=="World"])

growth = tax_growth.get(txs) # lo: 3%, med: 5%, hi: 10

value_taxes = [round((Base2020*((1+growth)**(x-2020))),2) for x in type_years]


Add CO2 Tax to the scenario

In [7]:
# In case you need to add the new unit to the platform
mp.add_unit("USD/tCO2")
tax_emission = []
for n in nodes:
    data = make_df(
        "tax_emission",
        node=n,
        type_year=type_years,
        type_tec="all",
        unit="USD/tCO2",
        type_emission="TCE",
        value=value_taxes,
        )
    tax_emission.append(data)

tax_emission = pd.concat(tax_emission)

scen.add_par("tax_emission", tax_emission)

In [8]:
scen.commit(comment="GENIE with"+lrn+"learning |"+txs+"carbon tax |"+rnd+"RND target")

In [9]:
scen.solve(gams_args =["--learningmode="+str(lrn_val)],solve_options={"scaind":-1})

In [10]:
scen.var("OBJ")["lvl"]

3051103.5

In [31]:
# List of technologies in electricity sector
etechs = ['coal_adv','coal_adv_ccs','coal_ppl','coal_ppl_u',
         'igcc','igcc_ccs','gas_cc','gas_cc_ccs','gas_ct',
         'gas_htfc','gas_ppl','bio_istig','bio_istig_ccs',
         'bio_ppl','geo_ppl','hydro_hc','hydro_lc','nuc_fbr',
         'nuc_hc','nuc_lc','foil_ppl','loil_cc','lio_ppl','oil_ppl',
         'SO2_scrub_ppl','solar_pv_ppl','csp_sm1_ppl','csp_sm3_ppl',
         'wind_ppf','wind_ppl','h2_bio','h2_bio_ccs','eth_bio',
         'eth_bio_ccs','liq_bio','liq_bio_ccs']
years = [2050,2100]
capacity = []

for y in years:
    capacity.append(scen.var("CAP", filters={'year_act':y,'technology':etechs})
                    .groupby('technology').sum().reindex(etechs)
                    .drop(columns=['year_vtg','year_act','mrg'])
                    .rename(columns={'lvl':y}).fillna(0)
                   )
capacity = pd.concat(capacity, axis=1)

with pd.ExcelWriter('output.xlsx',
                    mode='a') as writer:  
    capacity.to_excel(writer, sheet_name="CT"+txs+"-RNDn")

C:\Users\pratama\AppData\Local\Temp\ipykernel_2804\2877404961.py:15: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  .groupby('technology').sum().reindex(etechs)
C:\Users\pratama\AppData\Local\Temp\ipykernel_2804\2877404961.py:15: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  .groupby('technology').sum().reindex(etechs)


,2050,2100
technology,,
coal_adv,1752.734578,4673.995310
coal_adv_ccs,0.000000,0.000000
coal_ppl,265.357455,54.167697
coal_ppl_u,66.659371,0.000000
igcc,0.000000,0.000000
igcc_ccs,0.000000,0.000000
gas_cc,2069.005200,2675.312282
gas_cc_ccs,0.000000,0.000000
gas_ct,3003.824265,10774.718303


## Close the connection to the database

In [ ]:
mp.close_db()